In [45]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import GlobalAveragePooling2D,Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D,Activation
from keras.optimizers import Adam
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [27]:
BATCH_SIZE = 16
seeds = 41
im_shape = (250,250)

In [41]:
data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_generator = ImageDataGenerator(rescale=1./255)

train_generator = data_generator.flow_from_directory('dataset/train', target_size=im_shape, shuffle=True, seed=seeds,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="training")
validation_generator = val_data_generator.flow_from_directory('dataset/validation', target_size=im_shape, shuffle=False, seed=seeds,
                                                     class_mode='categorical', batch_size=BATCH_SIZE, subset="validation")

test_generator = test_generator.flow_from_directory('dataset/test', target_size=im_shape, shuffle=False, seed=seeds,
                                                     class_mode='categorical', batch_size=BATCH_SIZE)

nb_train_samples = train_generator.samples
nb_validation_samples = validation_generator.samples
nb_test_samples = test_generator.samples
classes = list(train_generator.class_indices.keys())
print('Classes: '+str(classes))
num_classes  = len(classes)

Found 87 images belonging to 1 classes.
Found 2 images belonging to 1 classes.
Found 30 images belonging to 1 classes.
Classes: ['Batagor']


In [42]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),activation='relu',input_shape=(im_shape[0],im_shape[1],3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 248, 248, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 124, 124, 16)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 122, 122, 32)      4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 61, 61, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 61, 61, 32)        0         
                                                                 
 conv2d_5 (Conv2D)           (None, 59, 59, 64)       

In [50]:
epochs = 12

history = model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // BATCH_SIZE,
    epochs=epochs,
    validation_data=validation_generator,
    verbose=1,
    validation_steps=nb_validation_samples // BATCH_SIZE)

Epoch 1/12
5/5 [==============================] - 4s 626ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/12
5/5 [==============================] - 4s 692ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/12
5/5 [==============================] - 3s 648ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/12
5/5 [==============================] - 4s 697ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/12
5/5 [==============================] - 5s 875ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/12
5/5 [==============================] - 4s 642ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/12
5/5 [==============================] - 3s 585ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/12
5/5 [==============================] - 3s 507ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/12
5/5 [==============================] - 3s 546ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/12
5/5 [==============================] - 3s 601ms/step - loss: 0.0000e+0

In [51]:
model.save("model.h5")

In [52]:
# Load the best saved model
from keras.models import load_model

#model = load_model('../input/classify-food-datas-models/model.h5')
model = load_model('model.h5')
score = model.evaluate_generator(validation_generator)

score = model.evaluate_generator(test_generator)
print('\nTest loss:', score[0])
print('Test accuracy:', score[1])

C:\Users\Acer\AppData\Local\Temp\ipykernel_13812\3492151276.py:6: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(validation_generator)
C:\Users\Acer\AppData\Local\Temp\ipykernel_13812\3492151276.py:8: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(test_generator)



Test loss: 0.0
Test accuracy: 1.0
